In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, HiveContext
from pyspark.sql.functions import *

In [2]:
!pip install PySPARQL==0.0.5

     |████████████████████████████████| 154 kB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 204.2 MB 61 kB/s s eta 0:00:01     |███████████████████████▊        | 151.6 MB 10.7 MB/s eta 0:00:05
     |████████████████████████████████| 198 kB 10.6 MB/s eta 0:00:01
     |████████████████████████████████| 231 kB 10.1 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 2.6 MB/s  eta 0:00:01
     |████████████████████████████████| 55 kB 5.3 MB/s  eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=2867ba7baef7085ad32aace76cce39dee7d54069345b34322818d79369dc11e8
  Stored in directory: /home/jovyan/.cache/pip/wheels/ea/21/84/970b03913d0d6a96ef51c34c878add0de9e4ecbb7c764ea21f
  Created wheel for rdflib-jsonld: filename=rdflib_jsonld-0.5.0-py2.py3-none-any.whl size=15347 sha256=36a98654f418354bbfa5ee5ebc18efe5ff314265c6391ad18f6fb1462d90c342
  Stored in directory: /home/jovyan/.cache/pip/wheels/3a/97/90/e133cbb98e34

In [3]:
from PySPARQL.Wrapper import PySPARQLWrapper

### Connection to Apache Spark

In [4]:
spark = (SparkSession
          .builder
          .appName("interfacing spark sql to hive metastore without configuration file")
          .config("hive.metastore.uris", "thrift://hive-metastore:9083") 
          .enableHiveSupport() 
          .getOrCreate())

### SPARQL query execution

In [5]:
sparql_endpoint = "http://jena-fuseki:3030/pizzads"

query = """
    PREFIX : <http://www.co-ode.org/ontologies/pizza/pizza.owl#>

    SELECT ?pizzaID ?outcome
    WHERE {

      ?pizzaType :suggestedTempLow ?tempLow; :suggestedDurationLow ?durLow;
                 :suggestedTempUp ?tempUp; :suggestedDurationUp ?durUp .

      SERVICE <http://ontop:8080/sparql> {
        ?pizzaID a ?pizzaType.
        ?pizzaID :temperature ?avgTemp; :start_cooking ?start; :end_cooking ?end.
      }

      BIND ((?end-?start) AS ?cookDuration)

      BIND( IF ((?avgTemp >= ?tempLow && ?avgTemp <= ?tempUp) &&
          (?cookDuration >= ?durLow && ?cookDuration <= ?durUp)
          ,"WELL_COOKED","ANOMALY") AS ?outcome)
    }
"""

wrapper = PySPARQLWrapper(spark, sparql_endpoint)
result = wrapper.query(query)
resultDF = result.dataFrame

pizzaID,outcome
http://www.co-ode.org/ontologies/pizza/pizza.owl#PZ789,ANOMALY
http://www.co-ode.org/ontologies/pizza/pizza.owl#PZ333,WELL_COOKED
http://www.co-ode.org/ontologies/pizza/pizza.owl#PZ222,WELL_COOKED
http://www.co-ode.org/ontologies/pizza/pizza.owl#PZ005,WELL_COOKED
http://www.co-ode.org/ontologies/pizza/pizza.owl#PZ456,WELL_COOKED
http://www.co-ode.org/ontologies/pizza/pizza.owl#PZ001,ANOMALY
http://www.co-ode.org/ontologies/pizza/pizza.owl#PZ789,ANOMALY
http://www.co-ode.org/ontologies/pizza/pizza.owl#PZ444,ANOMALY
http://www.co-ode.org/ontologies/pizza/pizza.owl#PZ333,WELL_COOKED
http://www.co-ode.org/ontologies/pizza/pizza.owl#PZ999,WELL_COOKED
http://www.co-ode.org/ontologies/pizza/pizza.owl#PZ111,WELL_COOKED
http://www.co-ode.org/ontologies/pizza/pizza.owl#PZ003,WELL_COOKED
http://www.co-ode.org/ontologies/pizza/pizza.owl#PZ004,ANOMALY
http://www.co-ode.org/ontologies/pizza/pizza.owl#PZ002,WELL_COOKED
http://www.co-ode.org/ontologies/pizza/pizza.owl#PZ888,WELL_COOKED

In [6]:
resultDF.show()  # Plot the results

+--------------------+-----------+
|             pizzaID|    outcome|
+--------------------+-----------+
|http://www.co-ode...|    ANOMALY|
|http://www.co-ode...|WELL_COOKED|
|http://www.co-ode...|WELL_COOKED|
|http://www.co-ode...|WELL_COOKED|
|http://www.co-ode...|WELL_COOKED|
|http://www.co-ode...|    ANOMALY|
|http://www.co-ode...|    ANOMALY|
|http://www.co-ode...|    ANOMALY|
|http://www.co-ode...|WELL_COOKED|
|http://www.co-ode...|WELL_COOKED|
|http://www.co-ode...|WELL_COOKED|
|http://www.co-ode...|WELL_COOKED|
|http://www.co-ode...|    ANOMALY|
|http://www.co-ode...|WELL_COOKED|
|http://www.co-ode...|WELL_COOKED|
|http://www.co-ode...|WELL_COOKED|
|http://www.co-ode...|WELL_COOKED|
|http://www.co-ode...|    ANOMALY|
|http://www.co-ode...|WELL_COOKED|
|http://www.co-ode...|    ANOMALY|
+--------------------+-----------+



In [7]:
# clean the result by removing the prefixes

df2 = resultDF.withColumn("pizzaID", regexp_replace('pizzaID','http://www.co-ode.org/ontologies/pizza/pizza.owl#',''))

In [8]:
df2.show()   # Plot the cleaned results

+-------+-----------+
|pizzaID|    outcome|
+-------+-----------+
|  PZ789|    ANOMALY|
|  PZ333|WELL_COOKED|
|  PZ222|WELL_COOKED|
|  PZ005|WELL_COOKED|
|  PZ456|WELL_COOKED|
|  PZ001|    ANOMALY|
|  PZ789|    ANOMALY|
|  PZ444|    ANOMALY|
|  PZ333|WELL_COOKED|
|  PZ999|WELL_COOKED|
|  PZ111|WELL_COOKED|
|  PZ003|WELL_COOKED|
|  PZ004|    ANOMALY|
|  PZ002|WELL_COOKED|
|  PZ888|WELL_COOKED|
|  PZ666|WELL_COOKED|
|  PZ777|WELL_COOKED|
|  PZ123|    ANOMALY|
|  PZ555|WELL_COOKED|
|  PZ222|    ANOMALY|
+-------+-----------+



### Persists the Spark Dataframe into a Spark table

In [9]:
df2.write.mode("overwrite").saveAsTable('pizzadb.analysis')